In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np  # For numerical computations
import pandas as pd  # For handling datasets (e.g., CSVs)
import tensorflow as tf  # Core deep learning framework
from matplotlib import pyplot as plt  # For plotting and visualization
from tensorflow import keras  # High-level API for neural networks
from tensorflow.keras.models import Sequential, load_model  # For building sequential models
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization  # Core CNN layers
from tensorflow.keras.optimizers import Adam  # Optimizer (if needed separately)
from tensorflow.keras.utils import to_categorical  # For one-hot encoding labels
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping
import optuna
from sklearn.model_selection import train_test_split
from keras_tuner import HyperModel
from keras_tuner.tuners import BayesianOptimization
import keras_tuner as kt

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session



X_train = np.load('/kaggle/input/street-view-house-numbers-svhn-dataset-numpy/X_train.npy')
Y_train = np.load('/kaggle/input/street-view-house-numbers-svhn-dataset-numpy/y_train.npy')

# Load test dataset
X_test = np.load('/kaggle/input/street-view-house-numbers-svhn-dataset-numpy/X_test.npy')
Y_test = np.load('/kaggle/input/street-view-house-numbers-svhn-dataset-numpy/y_test.npy')

#X_train = X_train / 255.0
#X_test = X_test / 255.0
X_train = X_train.transpose(3, 0, 1, 2)
X_test = X_test.transpose(3, 0, 1, 2)
X_train_resized = tf.image.resize(X_train, (32, 32))
X_test_resized = tf.image.resize(X_test, (32, 32))

Y_train_onehot = tf.keras.utils.to_categorical(Y_train, num_classes=10)
Y_test_onehot = tf.keras.utils.to_categorical(Y_test, num_classes=10)


def build_model(hp):
    model = Sequential()
    
    model.add(Conv2D(
        filters=hp.Int('filters', min_value=32, max_value=128, step=32),
        kernel_size=(3, 3),
        activation='relu',
        input_shape=(32, 32, 3)
    ))
    model.add(MaxPooling2D(2, 2))
    model.add(Flatten())
    
    model.add(Dense(
        hp.Int('units', min_value=64, max_value=512, step=64),
        activation='relu'
    ))
    
    model.add(Dense(10, activation='softmax'))
    
    model.compile(
        optimizer=hp.Choice('optimizer', values=['adam', 'sgd']),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model



tuner = kt.BayesianOptimization(
    build_model,
    objective='val_accuracy',  # Maximize validation accuracy
    max_trials=5,  # Number of different hyperparameter configurations to test
    executions_per_trial=1,  # Number of executions for each configuration
    directory='my_dir',  # Directory where results will be saved
    project_name='hyperparameter_tuning'
)



tuner.search(X_train_resized, Y_train_onehot, epochs=10, validation_data=(X_test_resized, Y_test_onehot))

# Get the best hyperparameters
best_hp = tuner.get_best_hyperparameters()[0]
print("Best hyperparameters:", best_hp)

# You can now build the best model and train it
best_model = tuner.hypermodel.build(best_hp)

# Train the model
best_model.fit(X_train_resized, Y_train_onehot, epochs=10, validation_data=(X_test_resized, Y_test_onehot))

# Save the best model
best_model.save('best_model.h5')


/kaggle/input/street-view-house-numbers-svhn-dataset-numpy/y_train.npy
/kaggle/input/street-view-house-numbers-svhn-dataset-numpy/y_test.npy
/kaggle/input/street-view-house-numbers-svhn-dataset-numpy/X_test.npy
/kaggle/input/street-view-house-numbers-svhn-dataset-numpy/X_train.npy
Reloading Tuner from my_dir/hyperparameter_tuning/tuner0.json

Search: Running Trial #2

Value             |Best Value So Far |Hyperparameter
96                |32                |filters
512               |256               |units
adam              |sgd               |optimizer

Epoch 1/10
2290/2290 ━━━━━━━━━━━━━━━━━━━━ 405s 176ms/step - accuracy: 0.5353 - loss: 1.4306 - val_accuracy: 0.8007 - val_loss: 0.7177
Epoch 2/10
2290/2290 ━━━━━━━━━━━━━━━━━━━━ 404s 176ms/step - accuracy: 0.8420 - loss: 0.5566 - val_accuracy: 0.8311 - val_loss: 0.5996
Epoch 3/10
2290/2290 ━━━━━━━━━━━━━━━━━━━━ 401s 175ms/step - accuracy: 0.8696 - loss: 0.4482 - val_accuracy: 0.8452 - val_loss: 0.5684
Epoch 4/10
2290/2290 ━━━━━━━━━━━━━━

In [7]:

X_train = np.load('/kaggle/input/street-view-house-numbers-svhn-dataset-numpy/X_train.npy')
Y_train = np.load('/kaggle/input/street-view-house-numbers-svhn-dataset-numpy/y_train.npy')

# Load test dataset
X_test = np.load('/kaggle/input/street-view-house-numbers-svhn-dataset-numpy/X_test.npy')
Y_test = np.load('/kaggle/input/street-view-house-numbers-svhn-dataset-numpy/y_test.npy')

#X_train = X_train / 255.0
#X_test = X_test / 255.0
X_train = X_train.transpose(3, 0, 1, 2)
X_test = X_test.transpose(3, 0, 1, 2)
X_train_resized = tf.image.resize(X_train, (32, 32))
X_test_resized = tf.image.resize(X_test, (32, 32))

Y_train = tf.keras.utils.to_categorical(Y_train, num_classes=10)
Y_test = tf.keras.utils.to_categorical(Y_test, num_classes=10)

'''
model = Sequential()

model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2)))

model.add(Flatten())

model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
          
model.add(Dense(10, activation='softmax'))

model.compile(optimizer='adam',
             loss='categorical_crossentropy',
             metrics=['accuracy'])

early_stopping = EarlyStopping(
    monitor='val_loss',   # Metric to monitor (validation loss)
    patience=5,           # Number of epochs to wait without improvement
    restore_best_weights=True  # Restore weights from the best epoch
)

model.fit(X_train_resized, Y_train, epochs=20, batch_size=32, callbacks=[early_stopping])

model.save('my_model_2.h5')
'''

model = load_model('my_model_2.h5')

loss, accuracy = model.evaluate(X_test, Y_test)

print(f"Test Loss: {loss}")
print(f"Test Accuracy: {accuracy}")


814/814 ━━━━━━━━━━━━━━━━━━━━ 8s 9ms/step - accuracy: 0.9176 - loss: 0.4413
Test Loss: 0.4191308915615082
Test Accuracy: 0.9193684458732605
